In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import xgboost

from functions import make_dataset

from functions import make_portfolio

from sklearn.utils import shuffle

import time

import pickle

In [24]:
prices_recent = pd.read_csv("simfin_data/us-shareprices-daily-3-2-2022-not-full/us-shareprices-daily.csv", sep = ";")

In [27]:
prices_recent = prices_recent.set_index(["Ticker", "Date"])

In [2]:
prices = pd.read_pickle("parsed_data/prices")

In [3]:
prices

SimFinId   Open    Low   High  Close  Adj. Close  Dividend  \
Ticker Date                                                                     
A      2011-05-20     45846  51.15  50.43  51.21  50.70       33.50       NaN   
       2011-05-23     45846  49.81  48.50  49.81  49.10       32.44       NaN   
       2011-05-24     45846  49.24  48.39  49.24  48.69       32.17       NaN   
       2011-05-25     45846  48.26  48.13  49.32  49.02       32.39       NaN   
       2011-05-26     45846  48.72  48.62  49.60  49.39       32.63       NaN   
...                     ...    ...    ...    ...    ...         ...       ...   
ZYNE   2021-09-13    901704   4.47   4.29   4.55   4.45        4.45       NaN   
       2021-09-14    901704   4.45   4.27   4.47   4.31        4.31       NaN   
       2021-09-15    901704   4.34   4.29   4.40   4.35        4.35       NaN   
       2021-09-16    901704   4.34   4.25   4.35   4.32        4.32       NaN   
       2021-09-17    901704   4.29   4.07   4.38   4.23        4.23       NaN   

                    Volume  Shares Outstanding  daily_return  return_40  
Ticker Date                                                              
A      2011-05-20  3350800         348045143.0           NaN  -7.416174  
       2011-05-23  4265800         348045143.0     -3.155819  -4.439919  
       2011-05-24  3437200         348045143.0     -0.835031  -3.635243  
       2011-05-25  3736200         348045143.0      0.677757  -4.345165  
       2011-05-26  3268500         348045143.0      0.754794  -6.053857  
...                    ...                 ...           ...        ...  
ZYNE   2021-09-13  1638339          41251537.0     -0.447427        NaN  
       2021-09-14   423426          41251537.0     -3.146067        NaN  
       2021-09-15   254110          41251537.0      0.928074        NaN  
       2021-09-16   317393          41251537.0     -0.689655        NaN  
       2021-09-17  2906024          41251537.0     -2.083333        NaN  

[4576018 rows x 11 columns]

In [ ]:
"""
we want to screen out anomalous data based on stock splits
just look at the change in shares outstanding


"""

In [4]:
Tickers = np.unique(prices.reset_index().set_index(["Ticker"]).index)

In [5]:
Dates = np.unique(prices.reset_index().set_index(["Date"]).index)

In [13]:
to_append = np.array([])
for i in Tickers:
    
    to_append = np.append(to_append, (prices.loc[i]["Shares Outstanding"].shift(-40) - prices.loc[i]["Shares Outstanding"] )/ prices.loc[i]["Shares Outstanding"] * 100)

In [14]:
prices["Shares Outstanding change 40"] = to_append

In [66]:
daily_return_argsort_highest = np.argsort(prices["daily_return"].values)[:(-np.isnan(prices["daily_return"].values).sum())][::-1]

In [78]:
daily_return_argsort_lowest = daily_return_argsort_highest[::-1]

In [94]:
return_40_argsort_highest = np.argsort(prices["return_40"].values)[:(-np.isnan(prices["return_40"].values).sum())][::-1]

In [112]:
return_40_highest = np.sort(prices["return_40"].values)[:(-np.isnan(prices["return_40"].values).sum())][::-1]

In [114]:
return_40_highest[250]

991.2698412698413

In [95]:
tickers = []
daily_return_argsort_highest_unique_ticker_indx = []
for i in return_40_argsort_highest[:1000]:
    ticker_name = prices.iloc[i].name[0]
    
    if ticker_name not in tickers:
        
        tickers.append(ticker_name)
        daily_return_argsort_highest_unique_ticker_indx.append(i)
        
        


In [97]:
len(daily_return_argsort_highest_unique_ticker_indx)

73

In [79]:
tickers = []
daily_return_argsort_lowest_unique_ticker_indx = []
for i in daily_return_argsort_lowest[:1000]:
    ticker_name = prices.iloc[i].name[0]
    
    if ticker_name not in tickers:
        
        tickers.append(ticker_name)
        daily_return_argsort_lowest_unique_ticker_indx.append(i)

In [80]:
len(daily_return_argsort_lowest_unique_ticker_indx)

469

In [68]:
len(tickers)

330

In [93]:
with open("CheckingStockSplits/blacklist", "wb") as f:
    pickle.dump(temp, f)

In [91]:
temp = []
for i in [blacklist, blacklist_all, blacklist_low, blacklist_all_low]:
    for j in i:
        temp.append([prices.iloc[j].name[0], prices.iloc[j].name[1], prices.iloc[j]["return_40"]])

In [99]:
blacklist = []
blacklist_all = []
good = []
len(good) + len(blacklist) + len(blacklist_all)

12

In [ ]:

for j in daily_return_argsort_highest_unique_ticker_indx[12:]:
    
    i = j
    
    fig, axs = plt.subplots(3, figsize = (20, 20))

    axs[0].plot(prices.iloc[i-50:i+50]["Close"].values, "b")
    axs[0].plot(prices.iloc[i-50:i+50]["Adj. Close"].values, "r")
    
    
    axs[1].plot(prices.iloc[i-50:i+50]["Volume"].values, "r")
    axs[2].plot(prices.iloc[i-50:i+50]["Shares Outstanding"].values, "g")

    plt.xlabel(prices.iloc[i].name[0] + "   " + prices.iloc[i].name[1])
    plt.show(block = False)
    
    cont = input("y or n")
    
    
    
    
    if cont == "nn":
        blacklist_all.append(i)
    
    elif cont == "n":
        blacklist.append(i)
    else:
        good.append(i)

In [101]:
temp

[['TXMD', '2011-10-31', 50.0],
 ['ALRN', '2017-06-29', 5.740740740740733],
 ['ALT', '2017-05-05', -59.03919089759798],
 ['PRPO', '2017-06-14', -32.857142857142854],
 ['DYNT', '2012-12-20', -25.13966480446927],
 ['PURE', '2012-08-15', -42.279411764705884],
 ['TGTX', '2012-02-06', -56.694312796208536],
 ['SQBG', '2012-09-12', 27.77777777777778],
 ['PAYS', '2011-11-14', -9.090909090909086],
 ['COT', '2012-11-02', 5.402010050251263],
 ['ISDR', '2011-05-23', -86.29032258064517],
 ['CPXX', '2016-03-15', 64.09395973154363],
 ['RSLS', '2021-01-20', -57.48792270531401],
 ['OTEL', '2013-05-28', -24.517212426532325],
 ['YOGAQ', '2019-07-24', -80.0],
 ['ROIA', '2020-06-16', -59.80139757263701],
 ['IMDZ', '2019-02-21', nan],
 ['LGCY', '2019-06-03', nan],
 ['SRNE', '2012-11-15', -4.3478260869565215],
 ['RLJE', '2015-09-18', -30.303030303030305],
 ['KDP', '2018-07-10', 3.4249662009914283],
 ['SIGM', '2018-08-15', -34.78260869565218],
 ['PAYS', '2011-11-11', 900.0],
 ['COT', '2012-11-01', 970.66666666

In [ ]:
"""
figure out where splits are being incorrectly represented

see where shares outstanding changed by x amount and stock price changed by 1/x

"""

In [ ]:
"""
if high change it is normally associated with higher than normal volumne for the day aswell
alot of these stocks also have really low volume
also can look at the shares outstanding change


there are 3 scenarios causing invalid data:
-stock splits not adjusted correctly
-

"""

In [3]:
Date1 = np.load("training_sets/Date1.npy", allow_pickle = True)

Date2 = np.load("training_sets/Date2.npy", allow_pickle = True)

Date3 = np.load("training_sets/Date3.npy", allow_pickle = True)

In [95]:
Date1 = np.load("validation_sets/Date1.npy", allow_pickle = True)

Date2 = np.load("validation_sets/Date2.npy", allow_pickle = True)

Date3 = np.load("validation_sets/Date3.npy", allow_pickle = True)

In [79]:
features3 = pd.read_pickle("training_sets/features3")

In [125]:
val3 = pd.read_pickle("validation_sets/val3")

In [126]:
val3 = val3.reset_index().set_index(["Ticker"])

In [127]:
Tickers3 = np.unique(val3.index)

In [128]:
len(Tickers3)

1910

In [32]:
with open("CheckingStockSplits/blacklist", "rb") as f:
    temp = pickle.load(f)

In [33]:
to_drop = np.array(temp)[:,0]

In [83]:
len(to_drop)

26

In [35]:
to_drop

array(['TXMD', 'ALRN', 'ALT', 'PRPO', 'DYNT', 'PURE', 'TGTX', 'SQBG',
       'PAYS', 'COT', 'ISDR', 'CPXX', 'RSLS', 'OTEL', 'YOGAQ', 'ROIA',
       'IMDZ', 'LGCY', 'SRNE', 'RLJE', 'KDP', 'SIGM', 'PAYS', 'COT',
       'ISDR', 'HROW', 'ANY', 'DS', 'YOGAQ', 'FMSA'], dtype='<U19')

In [36]:
to_drop = np.unique(to_drop)

In [37]:
len(to_drop)

26

In [84]:
to_drop

array(['ALRN', 'ALT', 'ANY', 'COT', 'CPXX', 'DS', 'DYNT', 'FMSA', 'HROW',
       'IMDZ', 'ISDR', 'KDP', 'LGCY', 'OTEL', 'PAYS', 'PRPO', 'PURE',
       'RLJE', 'ROIA', 'RSLS', 'SIGM', 'SQBG', 'SRNE', 'TGTX', 'TXMD',
       'YOGAQ'], dtype='<U19')

In [129]:
start_time = time.perf_counter()
for i in to_drop:
    
    try:
        val3 = val3.drop(i, axis = 0)

        print(time.perf_counter() - start_time)
        start_time = time.perf_counter()
    except:
        print(i + " was not removed")
        continue


0.29126019999989694
0.16290980000007949
0.14967599999999948
0.1652368000000024
CPXX was not removed
0.15780420000010054
0.16510879999987083
FMSA was not removed
0.18138790000011795
IMDZ was not removed
0.15200229999982184
0.15633840000009513
LGCY was not removed
0.1666439000000537
0.14976419999993595
0.1576716000001852
0.1479632999999012
RLJE was not removed
0.1512593999998444
0.14945190000003095
0.14704460000007202
0.1494394999999713
0.14850870000009309
0.1548453000000336
0.16784529999995357
0.14988240000002406


In [130]:
Tickers3_test = np.unique(val3.index)

In [131]:
len(Tickers3) - len(Tickers3_test)

21

In [132]:
Tickers3_test.dump("validation_sets/Tickers3.npy")

In [133]:
val3 = val3.reset_index().set_index(["Date", "Ticker"])

In [137]:
val3.to_pickle("validation_sets/val3")

In [138]:
del val3

In [46]:
len(Tickers1_test)

1682

In [134]:
val3

Close  ABAND_indicator_10  ABAND_indicator_20  \
Date       Ticker                                                   
2019-09-03 A        70.41           -0.052761            0.085382   
           AA       17.61           -0.040126           -0.178790   
           AAL      26.51            0.142317           -0.049685   
           AAMC     10.00            0.012520           -0.044174   
           AAOI      8.74           -0.184594           -0.300206   
...                   ...                 ...                 ...   
2020-01-07 ZTS     133.58            0.081846            0.504770   
           ZUMZ     34.18            0.154991            0.194739   
           ZUO      14.69            0.130864            0.106249   
           ZVO       2.01           -0.085246           -0.104159   
           ZYNE      5.56           -0.185496           -0.196833   

                   ABAND_indicator_40  ABAND_signal_10  ABAND_signal_20  \
Date       Ticker                                                         
2019-09-03 A                 0.067258              0.0              0.0   
           AA               -0.734812              0.0              0.0   
           AAL              -0.650533              0.0              0.0   
           AAMC             -0.185096              0.0              0.0   
           AAOI             -0.351083              0.0              0.0   
...                               ...              ...              ...   
2020-01-07 ZTS               0.946257              0.0              1.0   
           ZUMZ              0.453551              0.0              0.0   
           ZUO              -0.069081              0.0              0.0   
           ZVO              -0.184284              0.0              0.0   
           ZYNE             -0.296748              0.0              0.0   

                   ABAND_signal_40  MFV_40_10_indicator_sign_change  \
Date       Ticker                                                     
2019-09-03 A                   0.0                              0.0   
           AA                 -1.0                              0.0   
           AAL                -1.0                              0.0   
           AAMC                0.0                              0.0   
           AAOI                0.0                              0.0   
...                            ...                              ...   
2020-01-07 ZTS                 1.0                              0.0   
           ZUMZ                1.0                              0.0   
           ZUO                 0.0                              0.0   
           ZVO                 0.0                              0.0   
           ZYNE                0.0                              0.0   

                   APO_10_20  APO_10_40  ...  Class_binary_40_20  return_40  \
Date       Ticker                        ...                                  
2019-09-03 A        0.003715   0.001140  ...                 1.0   8.180656   
           AA      -0.038098  -0.101467  ...                 1.0  22.657581   
           AAL     -0.029216  -0.079761  ...                 1.0  16.635232   
           AAMC    -0.012705  -0.095848  ...                 0.0  35.500000   
           AAOI    -0.025269  -0.049087  ...                 1.0  12.585812   
...                      ...        ...  ...                 ...        ...   
2020-01-07 ZTS      0.018463   0.040729  ...                 1.0   4.476718   
           ZUMZ     0.018156   0.041363  ...                 1.0 -25.950848   
           ZUO     -0.001965  -0.007857  ...                 1.0 -17.971409   
           ZVO     -0.007208  -0.014819  ...                 1.0 -12.935323   
           ZYNE    -0.019867  -0.076569  ...                 1.0 -26.258993   

                   Class_binary_20_20  return_20  Class_binary_10_20  \
Date       Ticker                                                      
2019-09-03 A                      1.0   6.291720  

In [136]:
val3.loc[Date3]

Close  ABAND_indicator_10  ABAND_indicator_20  \
Date       Ticker                                                   
2019-09-03 A        70.41           -0.052761            0.085382   
           AA       17.61           -0.040126           -0.178790   
           AAL      26.51            0.142317           -0.049685   
           AAMC     10.00            0.012520           -0.044174   
           AAOI      8.74           -0.184594           -0.300206   
...                   ...                 ...                 ...   
2020-01-07 ZTS     133.58            0.081846            0.504770   
           ZUMZ     34.18            0.154991            0.194739   
           ZUO      14.69            0.130864            0.106249   
           ZVO       2.01           -0.085246           -0.104159   
           ZYNE      5.56           -0.185496           -0.196833   

                   ABAND_indicator_40  ABAND_signal_10  ABAND_signal_20  \
Date       Ticker                                                         
2019-09-03 A                 0.067258              0.0              0.0   
           AA               -0.734812              0.0              0.0   
           AAL              -0.650533              0.0              0.0   
           AAMC             -0.185096              0.0              0.0   
           AAOI             -0.351083              0.0              0.0   
...                               ...              ...              ...   
2020-01-07 ZTS               0.946257              0.0              1.0   
           ZUMZ              0.453551              0.0              0.0   
           ZUO              -0.069081              0.0              0.0   
           ZVO              -0.184284              0.0              0.0   
           ZYNE             -0.296748              0.0              0.0   

                   ABAND_signal_40  MFV_40_10_indicator_sign_change  \
Date       Ticker                                                     
2019-09-03 A                   0.0                              0.0   
           AA                 -1.0                              0.0   
           AAL                -1.0                              0.0   
           AAMC                0.0                              0.0   
           AAOI                0.0                              0.0   
...                            ...                              ...   
2020-01-07 ZTS                 1.0                              0.0   
           ZUMZ                1.0                              0.0   
           ZUO                 0.0                              0.0   
           ZVO                 0.0                              0.0   
           ZYNE                0.0                              0.0   

                   APO_10_20  APO_10_40  ...  Class_binary_40_20  return_40  \
Date       Ticker                        ...                                  
2019-09-03 A        0.003715   0.001140  ...                 1.0   8.180656   
           AA      -0.038098  -0.101467  ...                 1.0  22.657581   
           AAL     -0.029216  -0.079761  ...                 1.0  16.635232   
           AAMC    -0.012705  -0.095848  ...                 0.0  35.500000   
           AAOI    -0.025269  -0.049087  ...                 1.0  12.585812   
...                      ...        ...  ...                 ...        ...   
2020-01-07 ZTS      0.018463   0.040729  ...                 1.0   4.476718   
           ZUMZ     0.018156   0.041363  ...                 1.0 -25.950848   
           ZUO     -0.001965  -0.007857  ...                 1.0 -17.971409   
           ZVO     -0.007208  -0.014819  ...                 1.0 -12.935323   
           ZYNE    -0.019867  -0.076569  ...                 1.0 -26.258993   

                   Class_binary_20_20  return_20  Class_binary_10_20  \
Date       Ticker                                                      
2019-09-03 A                      1.0   6.291720  

In [ ]:
1213272